# Practical Work 5

## Students
- Flavia Pittet
- Fabio Vitali

## Ex1 - Gradient descent using matrix calculation

In [2]:
def hypothesis(theta,X):  #theta = 1xD, X = DxN, output 1xN
    return np.dot(theta,X)

def gradientDescent(X,y,learning_rate,num_epoch,verbose=False):
    N = X.shape[0]      # number of sample
    D = X.shape[1]      # number of dimensions
    theta = np.ones(D)  # init thetas to some values
    X_trans = X.transpose() # X_trans is DxN
    
    for i in range(0,num_epoch):
        h = hypothesis(theta,X_trans)  #N dimension
        loss = h-y                     #N dimension
        gradient = X_trans.dot(loss) * (1.0/N)  
        theta = theta - learning_rate * (1.0/N) * gradient
    return theta

a) Analyse the code above. It relies on matrix calculation. What type of gradient descent
does it implement ? Are you confident it is a correct implementation?

